# Elasticsearch python API overview 

In [51]:
import datetime

In [15]:
import warnings
from elasticsearch import Elasticsearch, RequestsHttpConnection
warnings.filterwarnings('ignore')

## Avant de commencer 

### Lancer elasticsearch avec docker 

Pour ce faire, on va run un cluster elastic dans un container. Si vous n'avez pas deja l'image elastic dans votre registery local il faut la pull du hub avec la commande suivante: 

```
docker pull docker.elastic.co/elasticsearch/elasticsearch:7.11.1
```

puis on run le container sur le port 9200 tel que: 

```
docker run -p 9200:9200 -p 9300:9300 -e "discovery.type=single-node" docker.elastic.co/elasticsearch/elasticsearch:7.11.1
```

### Lancer elasticsearch avec docker-compose 

On peut aussi lancer plusieurs noeud au sein d'un meme cluster avec docker-compose  tel que 

```Dockerfile 
version: '2.2'
services:
  es01:
    image: docker.elastic.co/elasticsearch/elasticsearch:7.11.1
    container_name: es01
    environment:
      - node.name=es01
      - cluster.name=es-docker-cluster
      - discovery.seed_hosts=es02,es03
      - cluster.initial_master_nodes=es01,es02,es03
      - bootstrap.memory_lock=true
      - "ES_JAVA_OPTS=-Xms512m -Xmx512m"
    ulimits:
      memlock:
        soft: -1
        hard: -1
    volumes:
      - data01:/usr/share/elasticsearch/data
    ports:
      - 9200:9200
    networks:
      - elastic
  es02:
    image: docker.elastic.co/elasticsearch/elasticsearch:7.11.1
    container_name: es02
    environment:
      - node.name=es02
      - cluster.name=es-docker-cluster
      - discovery.seed_hosts=es01,es03
      - cluster.initial_master_nodes=es01,es02,es03
      - bootstrap.memory_lock=true
      - "ES_JAVA_OPTS=-Xms512m -Xmx512m"
    ulimits:
      memlock:
        soft: -1
        hard: -1
    volumes:
      - data02:/usr/share/elasticsearch/data
    networks:
      - elastic
  es03:
    image: docker.elastic.co/elasticsearch/elasticsearch:7.11.1
    container_name: es03
    environment:
      - node.name=es03
      - cluster.name=es-docker-cluster
      - discovery.seed_hosts=es01,es02
      - cluster.initial_master_nodes=es01,es02,es03
      - bootstrap.memory_lock=true
      - "ES_JAVA_OPTS=-Xms512m -Xmx512m"
    ulimits:
      memlock:
        soft: -1
        hard: -1
    volumes:
      - data03:/usr/share/elasticsearch/data
    networks:
      - elastic

volumes:
  data01:
    driver: local
  data02:
    driver: local
  data03:
    driver: local

networks:
  elastic:
    driver: bridge
```

Plus d'info sur le doc [ici](https://www.elastic.co/guide/en/elasticsearch/reference/current/docker.html)

#### 🚧Attention à votre configuration Docker 🚧
Elastic demande beaucoup de ressource à votre docker (et donc à votre machine) il faut avoir au moins configurer 4GB de memoire que Docker peut utiliser. Vous pouvez aussi changer directement la configuration de la JVM des container avec le paramètre `ES_JAVA_OPTS=-Xms512m -Xmx512m` et le passer à `256m` ou bien `128m`. 


### 📟 Exercice [optionnel]

**Ecrire un fichier `docker-compose.yml` avec un service Elasticsearch sur le port 9200 (un seul noeud) et un service Kibana sur le port 5601 ainsi qu'un network elnet**

## Ping du container 

In [16]:
import requests
res = requests.get('http://localhost:9200?pretty')
print(res.content)

b'{\n  "name" : "330826bd0167",\n  "cluster_name" : "docker-cluster",\n  "cluster_uuid" : "yrB15y-NRrqvJ9PanktWAw",\n  "version" : {\n    "number" : "7.11.1",\n    "build_flavor" : "default",\n    "build_type" : "docker",\n    "build_hash" : "ff17057114c2199c9c1bbecc727003a907c0db7a",\n    "build_date" : "2021-02-15T13:44:09.394032Z",\n    "build_snapshot" : false,\n    "lucene_version" : "8.7.0",\n    "minimum_wire_compatibility_version" : "6.8.0",\n    "minimum_index_compatibility_version" : "6.0.0-beta1"\n  },\n  "tagline" : "You Know, for Search"\n}\n'


In [17]:
es = Elasticsearch('http://localhost:9200')

In [18]:
## Create, delete and verify index

#python
#create
es.indices.create(index="first_index",ignore=400)

#verify
print (es.indices.exists(index="first_index"))

#delete
print (es.indices.delete(index="first_index", ignore=[400,404]))

True
{'acknowledged': True}


In [19]:
## Insert documents

 
#documents to insert in the elasticsearch index "cities"
doc1 = {"city":"New Delhi", "country":"India"}
doc2 = {"city":"London", "country":"England"}
doc3 = {"city":"Los Angeles", "country":"USA"}

#Inserting doc1 in id=1
es.index(index="cities", doc_type="places", id=1, body=doc1)

#Inserting doc2 in id=2
es.index(index="cities", doc_type="places", id=2, body=doc2)

#Inserting doc3 in id=3
es.index(index="cities", doc_type="places", id=3, body=doc3)

{'_index': 'cities',
 '_type': 'places',
 '_id': '3',
 '_version': 3,
 'result': 'updated',
 '_shards': {'total': 2, 'successful': 1, 'failed': 0},
 '_seq_no': 8,
 '_primary_term': 2}

### 📟 Exercice [optionnel]
Trouver la fonction qui vérifie que votre index est bien crée.  

In [20]:
es.indices.exists(index="cities")

True

### Retrieve data with id : `get`

In [44]:
es.search(index="cities", filter_path=['hits.hits._*'])

{'hits': {'hits': [{'_index': 'cities',
    '_type': 'places',
    '_id': '1',
    '_score': 1.0,
    '_source': {'city': 'New Delhi', 'country': 'India'}},
   {'_index': 'cities',
    '_type': 'places',
    '_id': '2',
    '_score': 1.0,
    '_source': {'city': 'London', 'country': 'England'}},
   {'_index': 'cities',
    '_type': 'places',
    '_id': '3',
    '_score': 1.0,
    '_source': {'city': 'Los Angeles', 'country': 'USA'}}]}}

### 📟 Exercice [optionnel]
Afficher uniquement les informations ci-dessous à partir de la variable `res` 

In [41]:
res['hits']

{'hits': [{'_index': 'cities',
   '_type': 'places',
   '_id': '1',
   '_score': 1.0,
   '_source': {'city': 'New Delhi', 'country': 'India'}},
  {'_index': 'cities',
   '_type': 'places',
   '_id': '2',
   '_score': 1.0,
   '_source': {'city': 'London', 'country': 'England'}},
  {'_index': 'cities',
   '_type': 'places',
   '_id': '3',
   '_score': 1.0,
   '_source': {'city': 'Los Angeles', 'country': 'USA'}}]}

### Mapping

In [23]:
es.indices.get_mapping(index='cities')

{'cities': {'mappings': {'properties': {'city': {'type': 'text',
     'fields': {'keyword': {'type': 'keyword', 'ignore_above': 256}}},
    'country': {'type': 'text',
     'fields': {'keyword': {'type': 'keyword', 'ignore_above': 256}}}}}}}

More about mappings: https://www.elastic.co/guide/en/elasticsearch/reference/current/mapping.html

## Le endpoint `_search` et les `query`

Pour la suite des exemple assurez vous d'avoir importer les data via la `_bulk api`

In [45]:
res = es.search(index="cities", body={"query":{"match_all":{}}})
res

{'took': 0,
 'timed_out': False,
 '_shards': {'total': 1, 'successful': 1, 'skipped': 0, 'failed': 0},
 'hits': {'total': {'value': 3, 'relation': 'eq'},
  'max_score': 1.0,
  'hits': [{'_index': 'cities',
    '_type': 'places',
    '_id': '1',
    '_score': 1.0,
    '_source': {'city': 'New Delhi', 'country': 'India'}},
   {'_index': 'cities',
    '_type': 'places',
    '_id': '2',
    '_score': 1.0,
    '_source': {'city': 'London', 'country': 'England'}},
   {'_index': 'cities',
    '_type': 'places',
    '_id': '3',
    '_score': 1.0,
    '_source': {'city': 'Los Angeles', 'country': 'USA'}}]}}

In [46]:
res = es.search(index="movies", body={"query":{"match_all":{}}})
res

{'took': 116,
 'timed_out': False,
 '_shards': {'total': 1, 'successful': 1, 'skipped': 0, 'failed': 0},
 'hits': {'total': {'value': 4849, 'relation': 'eq'},
  'max_score': 1.0,
  'hits': [{'_index': 'movies',
    '_type': 'movie',
    '_id': '1',
    '_score': 1.0,
    '_source': {'fields': {'directors': ['Joseph Gordon-Levitt'],
      'release_date': '2013-01-18T00:00:00Z',
      'rating': 7.4,
      'genres': ['Comedy', 'Drama'],
      'image_url': 'http://ia.media-imdb.com/images/M/MV5BMTQxNTc3NDM2MF5BMl5BanBnXkFtZTcwNzQ5NTQ3OQ@@._V1_SX400_.jpg',
      'plot': 'A New Jersey guy dedicated to his family, friends, and church, develops unrealistic expectations from watching porn and works to find happiness and intimacy with his potential true love.',
      'title': 'Don Jon',
      'rank': 1,
      'running_time_secs': 5400,
      'actors': ['Joseph Gordon-Levitt',
       'Scarlett Johansson',
       'Julianne Moore'],
      'year': 2013},
     'id': 'tt2229499',
     'type': 'add'}},

### 📟 Exercice [optionnel]
Afficher uniquement les informations ci-dessous à partir de la variable `res` 

In [47]:
res = es.get(index="cities", doc_type="places",id=2)
res

{'_index': 'cities',
 '_type': 'places',
 '_id': '2',
 '_version': 3,
 '_seq_no': 7,
 '_primary_term': 2,
 'found': True,
 '_source': {'city': 'London', 'country': 'England'}}

In [48]:
res['_source']

{'city': 'London', 'country': 'England'}

### Affiner ces critères de recherche avec `_source`

In [53]:
es.search(index="movies", body={
  "_source": {
    "includes": [
      "*.title",
      "*.directors"
    ],
    "excludes": [
      "*.actors*",
      "*.genres"
    ]
  },
  "query": {
    "match": {
      "fields.directors": "George"
    }
  }
})

{'took': 1441,
 'timed_out': False,
 '_shards': {'total': 1, 'successful': 1, 'skipped': 0, 'failed': 0},
 'hits': {'total': {'value': 56, 'relation': 'eq'},
  'max_score': 5.019411,
  'hits': [{'_index': 'movies',
    '_type': 'movie',
    '_id': '3378',
    '_score': 5.019411,
    '_source': {'fields': {'directors': ['George Miller', 'George Ogilvie'],
      'title': 'Mad Max Beyond Thunderdome'}}},
   {'_index': 'movies',
    '_type': 'movie',
    '_id': '226',
    '_score': 4.6606607,
    '_source': {'fields': {'directors': ['George Lucas'],
      'title': 'Star Wars'}}},
   {'_index': 'movies',
    '_type': 'movie',
    '_id': '371',
    '_score': 4.6606607,
    '_source': {'fields': {'directors': ['George Lucas'],
      'title': 'Star Wars: Episode III - Revenge of the Sith'}}},
   {'_index': 'movies',
    '_type': 'movie',
    '_id': '469',
    '_score': 4.6606607,
    '_source': {'fields': {'directors': ['George Lucas'],
      'title': 'Star Wars: Episode I - The Phantom Menace

### Logique booléenne 

In [54]:
es.search(index="movies", body=
{
  "query": {
    "bool": {
      "must": [
        {
          "match": {
            "fields.directors": "George"
          }
        },
        {
          "match": {
            "fields.title": "Star Wars"
          }
        }
      ]
    }
  }
})


{'took': 3492,
 'timed_out': False,
 '_shards': {'total': 1, 'successful': 1, 'skipped': 0, 'failed': 0},
 'hits': {'total': {'value': 4, 'relation': 'eq'},
  'max_score': 17.335049,
  'hits': [{'_index': 'movies',
    '_type': 'movie',
    '_id': '226',
    '_score': 17.335049,
    '_source': {'fields': {'directors': ['George Lucas'],
      'release_date': '1977-05-25T00:00:00Z',
      'rating': 8.7,
      'genres': ['Action', 'Adventure', 'Fantasy', 'Sci-Fi'],
      'image_url': 'http://ia.media-imdb.com/images/M/MV5BMTU4NTczODkwM15BMl5BanBnXkFtZTcwMzEyMTIyMw@@._V1_SX400_.jpg',
      'plot': "Luke Skywalker joins forces with a Jedi Knight, a cocky pilot, a wookiee and two droids to save the universe from the Empire's world-destroying battle-station, while also attempting to rescue Princess Leia from the evil Darth Vader.",
      'title': 'Star Wars',
      'rank': 226,
      'running_time_secs': 7260,
      'actors': ['Mark Hamill', 'Harrison Ford', 'Carrie Fisher'],
      'year': 19

### Les critères : SHOULD / MUST

In [55]:
es.search(index="movies", body=
{
  "query": {
    "bool": {
      "must": [
                  { "match": { "fields.title": "Star Wars"}}
                  
      ],
      "must_not": { "match": { "fields.directors": "George Miller" }},
      "should": [
                  { "match": { "fields.title": "Star" }},
                  { "match": { "fields.directors": "George Lucas"}}
      ]
    }
  }
})

{'took': 263,
 'timed_out': False,
 '_shards': {'total': 1, 'successful': 1, 'skipped': 0, 'failed': 0},
 'hits': {'total': {'value': 25, 'relation': 'eq'},
  'max_score': 13.803417,
  'hits': [{'_index': 'movies',
    '_type': 'movie',
    '_id': '168',
    '_score': 13.803417,
    '_source': {'fields': {'directors': ['J.J. Abrams'],
      'release_date': '2015-01-01T00:00:00Z',
      'genres': ['Action', 'Adventure', 'Fantasy', 'Sci-Fi'],
      'plot': 'A continuation of the saga created by George Lucas.',
      'title': 'Star Wars: Episode VII',
      'rank': 168,
      'year': 2015,
      'actors': ['Mark Hamill', 'Harrison Ford', 'Carrie Fisher']},
     'id': 'tt2488496',
     'type': 'add'}},
   {'_index': 'movies',
    '_type': 'movie',
    '_id': '128',
    '_score': 11.640135,
    '_source': {'fields': {'directors': ['J.J. Abrams'],
      'release_date': '2009-04-06T00:00:00Z',
      'rating': 8,
      'genres': ['Action', 'Adventure', 'Sci-Fi'],
      'image_url': 'http://ia.

### Filtrer ses query avec `filter` 

On cherche ici les recettes avec un ingrédient de type `parmesan` sans ingrédient `tuna` en filtrant les recettes avec un temps de préparation inférieur ou egale à 15minutes.  

In [56]:
es.search(index="receipe", body={
  "query": {
    "bool": {
      "must": [
        {
          "match": {
            "ingredients.name": "parmesan"
          }
        }
      ], 
      "must_not": [
        {
          "match": {
            "ingredients.name": "tuna"
          }
        }
      ], 
      "filter": [
        {
          "range":{
            "preparation_time_minutes": {
              "lte":15
            }
          }
        }
        ]
    }
  }
})



{'took': 679,
 'timed_out': False,
 '_shards': {'total': 1, 'successful': 1, 'skipped': 0, 'failed': 0},
 'hits': {'total': {'value': 2, 'relation': 'eq'},
  'max_score': 1.379573,
  'hits': [{'_index': 'receipe',
    '_type': '_doc',
    '_id': '1',
    '_score': 1.379573,
    '_source': {'title': 'Fast and Easy Pasta With Blistered Cherry Tomato Sauce',
     'description': "Cherry tomatoes are almost always sweeter, riper, and higher in pectin than larger tomatoes at the supermarket. All of these factors mean that cherry tomatoes are fantastic for making a rich, thick, flavorful sauce. Even better: It takes only four ingredients and about 10 minutes, start to finish—less time than it takes to cook the pasta you're gonna serve it with.",
     'preparation_time_minutes': 12,
     'servings': {'min': 4, 'max': 6},
     'steps': ['Place pasta in a large skillet or sauté pan and cover with water and a big pinch of salt. Bring to a boil over high heat, stirring occasionally. Boil until jus

### Recherche avec un prefix 

Les query de type `prefix` permettent de trouver tout les termes commencant par le(s) caractère(s) correspondant.  

In [57]:
es.search(index="cities", body={"query": {"prefix" : { "city" : "l" }}})

{'took': 3441,
 'timed_out': False,
 '_shards': {'total': 1, 'successful': 1, 'skipped': 0, 'failed': 0},
 'hits': {'total': {'value': 2, 'relation': 'eq'},
  'max_score': 1.0,
  'hits': [{'_index': 'cities',
    '_type': 'places',
    '_id': '2',
    '_score': 1.0,
    '_source': {'city': 'London', 'country': 'England'}},
   {'_index': 'cities',
    '_type': 'places',
    '_id': '3',
    '_score': 1.0,
    '_source': {'city': 'Los Angeles', 'country': 'USA'}}]}}

### Rechercher avec des regex 

In [58]:
#tout afficher 
es.search(index="cities", body={"query": {"regexp" : { "city" : ".*" }}})

{'took': 400,
 'timed_out': False,
 '_shards': {'total': 1, 'successful': 1, 'skipped': 0, 'failed': 0},
 'hits': {'total': {'value': 4, 'relation': 'eq'},
  'max_score': 1.0,
  'hits': [{'_index': 'cities',
    '_type': 'places',
    '_id': '1',
    '_score': 1.0,
    '_source': {'city': 'New Delhi', 'country': 'India'}},
   {'_index': 'cities',
    '_type': 'places',
    '_id': '2',
    '_score': 1.0,
    '_source': {'city': 'London', 'country': 'England'}},
   {'_index': 'cities',
    '_type': 'places',
    '_id': '3',
    '_score': 1.0,
    '_source': {'city': 'Los Angeles', 'country': 'USA'}},
   {'_index': 'cities',
    '_type': 'places',
    '_id': '4',
    '_score': 1.0,
    '_source': {'city': 'Sydney',
     'country': 'Australia',
     'datetime': '2019-04-19T18:20:00'}}]}}

In [59]:
#afficher les cities qui commencent par L
es.search(index="cities", body={"query": {"regexp" : { "city" : "l.*" }}})

{'took': 631,
 'timed_out': False,
 '_shards': {'total': 1, 'successful': 1, 'skipped': 0, 'failed': 0},
 'hits': {'total': {'value': 2, 'relation': 'eq'},
  'max_score': 1.0,
  'hits': [{'_index': 'cities',
    '_type': 'places',
    '_id': '2',
    '_score': 1.0,
    '_source': {'city': 'London', 'country': 'England'}},
   {'_index': 'cities',
    '_type': 'places',
    '_id': '3',
    '_score': 1.0,
    '_source': {'city': 'Los Angeles', 'country': 'USA'}}]}}

In [60]:
#afficher les cities qui commencent par L et terminent par n 
es.search(index="cities", body={"query": {"regexp" : { "city" : "l.*n" }}})

{'took': 1,
 'timed_out': False,
 '_shards': {'total': 1, 'successful': 1, 'skipped': 0, 'failed': 0},
 'hits': {'total': {'value': 1, 'relation': 'eq'},
  'max_score': 1.0,
  'hits': [{'_index': 'cities',
    '_type': 'places',
    '_id': '2',
    '_score': 1.0,
    '_source': {'city': 'London', 'country': 'England'}}]}}

### Agregation 

In [61]:
#agregation simple -> movies/years
res = es.search(index="movies",body={"aggs" : {
    "nb_par_annee" : {
        "terms" : {"field" : "fields.year"}
}}})
res['aggregations']

{'nb_par_annee': {'doc_count_error_upper_bound': 0,
  'sum_other_doc_count': 2192,
  'buckets': [{'key': 2013, 'doc_count': 448},
   {'key': 2012, 'doc_count': 404},
   {'key': 2011, 'doc_count': 308},
   {'key': 2009, 'doc_count': 253},
   {'key': 2010, 'doc_count': 249},
   {'key': 2008, 'doc_count': 207},
   {'key': 2006, 'doc_count': 204},
   {'key': 2007, 'doc_count': 200},
   {'key': 2005, 'doc_count': 170},
   {'key': 2014, 'doc_count': 152}]}}

In [62]:
#agregation et stats simple -> moyennes des raitings 
res = es.search(index="movies",body={"aggs" : {
    "note_moyenne" : {
        "avg" : {"field" : "fields.rating"}
}}})
res['aggregations']

{'note_moyenne': {'value': 6.387107691895831}}

In [63]:
#agregation et stats simple -> stats basiques raitings/years
res = es.search(index="movies",body={"aggs" : {
    "group_year" : {
        "terms" : { "field" : "fields.year" },
        "aggs" : {
            "note_moyenne" : {"avg" : {"field" : "fields.rating"}},
            "note_min" : {"min" : {"field" : "fields.rating"}},
            "note_max" : {"max" : {"field" : "fields.rating"}}
        }
}}})
res["aggregations"]

{'group_year': {'doc_count_error_upper_bound': 0,
  'sum_other_doc_count': 2192,
  'buckets': [{'key': 2013,
    'doc_count': 448,
    'note_max': {'value': 8.699999809265137},
    'note_moyenne': {'value': 5.962700002789497},
    'note_min': {'value': 2.5}},
   {'key': 2012,
    'doc_count': 404,
    'note_max': {'value': 8.600000381469727},
    'note_moyenne': {'value': 5.961786593160322},
    'note_min': {'value': 2.4000000953674316}},
   {'key': 2011,
    'doc_count': 308,
    'note_max': {'value': 8.5},
    'note_moyenne': {'value': 6.114285714440531},
    'note_min': {'value': 1.7000000476837158}},
   {'key': 2009,
    'doc_count': 253,
    'note_max': {'value': 8.399999618530273},
    'note_moyenne': {'value': 6.268774692248921},
    'note_min': {'value': 2.700000047683716}},
   {'key': 2010,
    'doc_count': 249,
    'note_max': {'value': 8.800000190734863},
    'note_moyenne': {'value': 6.239759046868627},
    'note_min': {'value': 1.7999999523162842}},
   {'key': 2008,
    'd

### 📟 Exercice [optionnel]
Tester d'autres requetes

### Datetime agrégation 

Pour illuster l'agrégation par datetime on va créer un index `travel` et utiliser des data de type :
```
doc1 = {"city":"Bangalore", "country":"India","datetime": datetime.datetime(2018,1,1,10,20,0)} 
```

In [64]:
#specify mapping and create index 
if es.indices.exists(index="travel"):
    es.indices.delete(index="travel", ignore=[400,404])

settings = {
    "settings": {
        "number_of_shards": 2,
        "number_of_replicas": 1
    },
    "mappings": {
            "properties": {
                "city": {
                        "type": "text",
                        "fields": {
                            "keyword": {
                                "type": "keyword",
                                "ignore_above": 256
                            }
                        }
                    },
                    "country": {
                        "type": "text",
                        "fields": {
                            "keyword": {
                                "type": "keyword",
                                "ignore_above": 256
                            }
                        }
                    },
                    "datetime": {
                        "type": "date",
                    }
        }
     }
}
es.indices.create(index="travel", ignore=400, body=settings)

{'acknowledged': True, 'shards_acknowledged': True, 'index': 'travel'}

In [65]:
import datetime
doc1 = {"city":"Bangalore", "country":"India","datetime": datetime.datetime(2018,1,1,10,20,0)} #datetime format: yyyy,MM,dd,hh,mm,ss
doc2 = {"city":"London", "country":"England","datetime": datetime.datetime(2018,1,2,22,30,0)}
doc3 = {"city":"Los Angeles", "country":"USA","datetime": datetime.datetime(2018,4,19,18,20,0)}
es.index(index="travel", id=1, body=doc1)
es.index(index="travel", id=2, body=doc2)
es.index(index="travel", id=3, body=doc3)

{'_index': 'travel',
 '_type': '_doc',
 '_id': '3',
 '_version': 1,
 'result': 'created',
 '_shards': {'total': 2, 'successful': 1, 'failed': 0},
 '_seq_no': 2,
 '_primary_term': 1}

In [66]:
es.indices.get_mapping(index='travel')

{'travel': {'mappings': {'properties': {'city': {'type': 'text',
     'fields': {'keyword': {'type': 'keyword', 'ignore_above': 256}}},
    'country': {'type': 'text',
     'fields': {'keyword': {'type': 'keyword', 'ignore_above': 256}}},
    'datetime': {'type': 'date'}}}}}

In [67]:
es.search(index="travel", 
body={"from": 0, 
        "size": 0, 
            "query": {"match_all": {}}, 
                "aggs": {
                        "country": {
                            "date_histogram": {"field": "datetime", "calendar_interval": "year"
                        }
                       }
                      }
                    })

{'took': 2359,
 'timed_out': False,
 '_shards': {'total': 2, 'successful': 2, 'skipped': 0, 'failed': 0},
 'hits': {'total': {'value': 1, 'relation': 'eq'},
  'max_score': None,
  'hits': []},
 'aggregations': {'country': {'buckets': [{'key_as_string': '2018-01-01T00:00:00.000Z',
     'key': 1514764800000,
     'doc_count': 1}]}}}

### 📟 Exercice [optionnel]
Créer le document suivant et inserer le en base afin de rafficher l'histogramme precedent, dite ce qui à changer.
```
doc4 = {"city":"Sydney", "country":"Australia","datetime":datetime.datetime(2019,4,19,18,20,0)}
```

In [69]:
doc4 = {"city":"Sydney", "country":"Australia","datetime":datetime.datetime(2019,4,19,18,20,0)}

es.index(index="travel",id=4, body=doc4)

{'_index': 'travel',
 '_type': '_doc',
 '_id': '4',
 '_version': 1,
 'result': 'created',
 '_shards': {'total': 2, 'successful': 1, 'failed': 0},
 '_seq_no': 0,
 '_primary_term': 1}

{'_id': '4',
 '_index': 'travel',
 '_primary_term': 1,
 '_seq_no': 0,
 '_shards': {'failed': 0, 'successful': 1, 'total': 2},
 '_type': '_doc',
 '_version': 1,
 'result': 'created'}

In [ ]:
es.search_shards(index="travel",body=)

{'_shards': {'failed': 0, 'skipped': 0, 'successful': 2, 'total': 2},
 'aggregations': {'country': {'buckets': []}},
 'hits': {'hits': [],
  'max_score': None,
  'total': {'relation': 'eq', 'value': 0}},
 'timed_out': False,
 'took': 5}

## Search text introduction : endpoint `_analyze`

### Construire un Analyzer
Avant de commencer cette partie assurez vous d'avoir créer un `french analyzer` dans elasticsearch. 
Ci joint l'exemple d'analyzer francais vu dans le cour : 
```json
PUT french
{
  "settings": {
    "analysis": { 
      "filter": {
        "french_elision": {
          "type": "elision",
          "articles_case": true,
          "articles": ["l", "m", "t", "qu", "n", "s", "j", "d", "c", "jusqu", "quoiqu", "lorsqu", "puisqu"]
        },
        "french_synonym": {
          "type": "synonym",
          "ignore_case": true,
          "expand": true,
          "synonyms": [
            "réviser, étudier, bosser",
            "mayo, mayonnaise",
            "grille, toaste"
          ]
        },
        "french_stemmer": {
          "type": "stemmer",
          "language": "light_french"
        }
      },
      "analyzer": {
        "french_heavy": {
          "tokenizer": "icu_tokenizer",
          "filter": [
            "french_elision",
            "icu_folding",
            "french_synonym",
            "french_stemmer"
          ]
        },
        "french_light": {
          "tokenizer": "icu_tokenizer",
          "filter": [
            "french_elision",
            "icu_folding"
          ]
        }
      }
    }
  }
}
```

🤓 Assurer vous d'installer le pluging qui contient `icu_tokenizer` avant sinon vous allez avoir une erreur. 

In [27]:
doc1 = {"text" : "Une phrase en français :) ..."}
es.index(index="french", id=1, body=doc1)

{'_id': '1',
 '_index': 'french',
 '_primary_term': 1,
 '_seq_no': 2,
 '_shards': {'failed': 0, 'successful': 1, 'total': 2},
 '_type': '_doc',
 '_version': 3,
 'result': 'updated'}

In [28]:
es.indices.analyze(index="french",body={
  "text" : "Je dois bosser pour mon QCM sinon je vais avoir une sale note :( ..."
})

{'tokens': [{'end_offset': 2,
   'position': 0,
   'start_offset': 0,
   'token': 'je',
   'type': '<ALPHANUM>'},
  {'end_offset': 7,
   'position': 1,
   'start_offset': 3,
   'token': 'dois',
   'type': '<ALPHANUM>'},
  {'end_offset': 14,
   'position': 2,
   'start_offset': 8,
   'token': 'bosser',
   'type': '<ALPHANUM>'},
  {'end_offset': 19,
   'position': 3,
   'start_offset': 15,
   'token': 'pour',
   'type': '<ALPHANUM>'},
  {'end_offset': 23,
   'position': 4,
   'start_offset': 20,
   'token': 'mon',
   'type': '<ALPHANUM>'},
  {'end_offset': 27,
   'position': 5,
   'start_offset': 24,
   'token': 'qcm',
   'type': '<ALPHANUM>'},
  {'end_offset': 33,
   'position': 6,
   'start_offset': 28,
   'token': 'sinon',
   'type': '<ALPHANUM>'},
  {'end_offset': 36,
   'position': 7,
   'start_offset': 34,
   'token': 'je',
   'type': '<ALPHANUM>'},
  {'end_offset': 41,
   'position': 8,
   'start_offset': 37,
   'token': 'vais',
   'type': '<ALPHANUM>'},
  {'end_offset': 47,
   'p

### 📟 Exercice [optionnel]
Ajouter une fonctionnalités de reconnaissance de smiley à votre analyzer, de sorte qu'il fasse le lien suivant : 
```
:) -> _content_
:( -> _triste_
```
Faite ensuite une requete en python sur le document ci-dessous : 
```json
{
     "text" : "Je dois bosser pour mon QCM sinon je vais avoir une sale note :( ..."
}
```

In [74]:
res = es.indices.analyze(
index="french",
body={
  "settings": {
    "analysis": {
      "analyzer": {
        "my_analyzer": {
          "tokenizer": "keyword",
          "filter": [
            "my_pattern_replace_filter"
          ]
        }
      },
      "filter": {
        "my_pattern_replace_filter": {
          "type": "pattern_replace",
          "pattern": ":(",
          "replacement": "_triste_",
          "text" : "Je dois bosser pour mon QCM sinon je vais avoir une sale note :( ...",
        }
      }
    }
  }
}
)

RequestError: RequestError(400, 'x_content_parse_exception', '[1:2] [analyze_request] unknown field [settings]')

{'tokens': [{'end_offset': 2,
   'position': 0,
   'start_offset': 0,
   'token': 'je',
   'type': '<ALPHANUM>'},
  {'end_offset': 7,
   'position': 1,
   'start_offset': 3,
   'token': 'dois',
   'type': '<ALPHANUM>'},
  {'end_offset': 14,
   'position': 2,
   'start_offset': 8,
   'token': 'bosser',
   'type': '<ALPHANUM>'},
  {'end_offset': 19,
   'position': 3,
   'start_offset': 15,
   'token': 'pour',
   'type': '<ALPHANUM>'},
  {'end_offset': 23,
   'position': 4,
   'start_offset': 20,
   'token': 'mon',
   'type': '<ALPHANUM>'},
  {'end_offset': 27,
   'position': 5,
   'start_offset': 24,
   'token': 'qcm',
   'type': '<ALPHANUM>'},
  {'end_offset': 33,
   'position': 6,
   'start_offset': 28,
   'token': 'sinon',
   'type': '<ALPHANUM>'},
  {'end_offset': 36,
   'position': 7,
   'start_offset': 34,
   'token': 'je',
   'type': '<ALPHANUM>'},
  {'end_offset': 41,
   'position': 8,
   'start_offset': 37,
   'token': 'vais',
   'type': '<ALPHANUM>'},
  {'end_offset': 47,
   'p